In [1]:
from datasets import load_dataset, concatenate_datasets
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load datasets
eng_sarc_dataset = load_dataset(path='raquiba/Sarcasm_News_Headline')
ar_sarc_dataset = load_dataset(path='ar_sarcasm', ignore_verifications=True)
kor_sarc_dataset = load_dataset(path='kor_sarcasm')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/raquiba--Sarcasm_News_Headline-2c0b20e2ac20e61f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8437 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2110 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/301 [00:00<?, ? examples/s]

Dataset kor_sarcasm downloaded and prepared to /root/.cache/huggingface/datasets/kor_sarcasm/default/1.1.0/00d38c200d4d563ed94efb9ff4ca119ded94fe3cdf1e381ed95274de0a9d59f0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
### process datasets
eng_sarc_dataset = eng_sarc_dataset.remove_columns(column_names=['article_link'])
eng_sarc_dataset = eng_sarc_dataset.rename_columns({"is_sarcastic":"label"})

ar_sarc_dataset = ar_sarc_dataset.remove_columns(column_names=['dialect', 'sentiment', 'original_sentiment', 'source'])
ar_sarc_dataset = ar_sarc_dataset.rename_columns({"tweet":"headline", "sarcasm":"label"})

kor_sarc_dataset = kor_sarc_dataset.rename_columns({"tokens":"headline"})

In [4]:
model_ckpt = 'distilbert-base-uncased'

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [18]:
# non_sarc_ar = ar_sarc_dataset.filter(lambda x: x['label'] == 0).select(range(2600))
# sarc_ar = ar_sarc_dataset.filter(lambda x: x['label'] == 1)
# ar_sarc_dataset = concatenate_datasets([sarc_ar, non_sarc_ar])

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [6]:
# tokenize datasets using distilbert pretrained tokenizer
encoded_eng_dataset = eng_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512, padding="max_length", truncation=True), batched=True).shuffle()
encoded_ar_dataset = ar_sarc_dataset.map(lambda sample: tokenizer(sample['headline'],  max_length=512, padding="max_length", truncation=True), batched=True).shuffle()
encoded_kor_dataset = kor_sarc_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512,  padding="max_length", truncation=True), batched=True).shuffle()

  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
from transformers import AutoModelForSequenceClassification, DistilBertConfig
import torch 

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = DistilBertConfig(seq_classif_dropout=0.3)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.we

In [9]:
from transformers import TrainingArguments, Trainer

In [10]:
model_name = f"{model_ckpt}-multiling-sarcasm"
training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=3,
                                 learning_rate=2e-5,
                                 per_device_train_batch_size=32,
                                 per_device_eval_batch_size=32,
                                 evaluation_strategy='epoch',
                                 weight_decay=0.01,
                                 log_level='error',
                                 push_to_hub=False,
                                 report_to='none')

In [11]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {"f1": f1, "acc": accuracy}

In [26]:
# encoded_eng_dataset_rdy = encoded_eng_dataset.train_test_split(test_size=0.3)

In [12]:
trainer_eng = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_eng_dataset['train'],
                 eval_dataset=encoded_eng_dataset['test'])
trainer_eng.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.143300,0.948185,0.948182
2,0.282500,0.080993,0.973159,0.973155
3,0.139600,0.061058,0.980939,0.980943


TrainOutput(global_step=1344, training_loss=0.18107889947437106, metrics={'train_runtime': 2842.3084, 'train_samples_per_second': 30.207, 'train_steps_per_second': 0.473, 'total_flos': 1.1373253446408192e+16, 'train_loss': 0.18107889947437106, 'epoch': 3.0})

In [13]:
trainer_ar = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_ar_dataset['train'],
                 eval_dataset=encoded_ar_dataset['test'])
trainer_ar.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.398996,0.764296,0.837441
2,No log,0.387086,0.763160,0.836967
3,No log,0.382767,0.767177,0.837915


TrainOutput(global_step=396, training_loss=0.39276758829752606, metrics={'train_runtime': 674.8913, 'train_samples_per_second': 37.504, 'train_steps_per_second': 0.587, 'total_flos': 3352882327382016.0, 'train_loss': 0.39276758829752606, 'epoch': 3.0})

In [ ]:
#encoded_kor_dataset_rdy = encoded_kor_dataset.train_test_split(test_size=0.25)

In [14]:
trainer_kor = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_kor_dataset['train'],
                 eval_dataset=encoded_kor_dataset['test'])
trainer_kor.train()

Epoch,Training Loss,Validation Loss,F1,Acc
1,No log,0.557135,0.704437,0.704319
2,No log,0.541804,0.697795,0.697674
3,No log,0.536978,0.723375,0.724252


TrainOutput(global_step=423, training_loss=0.531447282074191, metrics={'train_runtime': 660.6333, 'train_samples_per_second': 40.87, 'train_steps_per_second': 0.64, 'total_flos': 3576619763712000.0, 'train_loss': 0.531447282074191, 'epoch': 3.0})

In [ ]:
eng_sarc_dataset = eng_sarc_dataset.cast(kor_sarc_dataset.features)
ar_sarc_dataset  = ar_sarc_dataset.cast(kor_sarc_dataset.features)

In [ ]:
all_dataset = concatenate_datasets([eng_sarc_dataset, ar_sarc_dataset, kor_sarc_dataset])

In [ ]:
encoded_all_dataset = all_dataset.map(lambda sample: tokenizer(sample['headline'], max_length=512, padding="max_length", truncation=True), batched=True).shuffle()

In [ ]:
encoded_all_dataset_rdy = encoded_all_dataset.train_test_split(test_size=0.25)

In [ ]:
trainer = Trainer(model=model,
                 args=training_args,
                 compute_metrics=compute_metrics,
                 train_dataset=encoded_all_dataset_rdy['train'],
                 eval_dataset=encoded_all_dataset_rdy['test'])
trainer.train()

In [ ]:
import pandas as pd 

df_log = pd.DataFrame(trainer.state.log_history)

In [ ]:
df_log.dropna(subset=['loss']).reset_index()['loss'].transpose()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df_log.dropna(subset=['loss']).reset_index()['loss'])

In [ ]:
encoded_eng_dataset.filter(lambda x: x['label'] == 1)

In [ ]:
encoded_eng_dataset_rdy['train'].filter(lambda x: x['label'] == 1)